# SCohenLab 2D BATCH Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## ❶ GOAL SETTING ✍

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_ ✅
- robust inference of subcellular objects:
  -  #### 1️⃣. [infer NUCLEI ](./01_infer_nuclei.ipynb) 
  -  #### 2️⃣. [Infer SOMA](./02_infer_soma.ipynb) (🚨🚨🚨🚨 Steps 3-9 depend on establishing a good solution here.)
  -  #### 3️⃣. [Infer CYTOSOL](./03_infer_cytosol.ipynb)
  -  #### 4️⃣. [Infer LYSOSOMES](./04_infer_lysosome.ipynb)
  -  #### 5️⃣. [Infer MITOCHONDRIA](./05_infer_mitochondria.ipynb)
  -  #### 6️⃣. [Infer GOLGI complex](./06_golgi.ipynb)
  -  #### 7️⃣. [Infer PEROXISOMES](./07_peroxisome.ipynb)
  -  #### 8️⃣. [Infer ENDOPLASMIC RETICULUM ](./08_endoplasmic_reticulum.ipynb)
  -  #### 9️⃣. [Infer LB](./09_lipid_bodies.ipynb) 






## ❷ DATA CREATION
> METHODS:📚📚
> 
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA 🏺 (artifacts)
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## ❸. IMAGE PROCESSING ⚙️🩻🔬
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A
# IMPORTS

import  all nescessary packages

we are using `napari` for visualization, and `scipy` `ndimage` and `skimage` for analyzing the image files.  The underlying data format are `numpy` `ndarrays` and tools from  Allen Institute for Cell Science.


In [3]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice, edge_preserving_smoothing_3d )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles import *

from infer_subc_2d.constants import TEST_IMG_N

%load_ext autoreload
%autoreload 2

test_img_n = TEST_IMG_N

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [4]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)


test_img_name = img_file_list[test_img_n]


In [5]:

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [6]:

ch_to_agg = (1,2,3,4,5,6)
nuc_ch = 0
optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 4,
    "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
    "nuc_ch" : 0,
    "optimal_Z": optimal_Z,
})


save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


img_2D = img_data[:,[optimal_Z],:,:].copy()


## get the inferred nuclei object

(takes < 1 sec)

In [21]:

###################
# INPUT NU
###################


raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [22]:

###################
# GET SOMA
###################

raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )

SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, out_p) 
CY_object =  infer_CYTOSOL(SO_object, NU_object) 

nuclei  = apply_mask(raw_nuclei, SO_object)
NU_object  = apply_mask(NU_object, SO_object)


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


Builde the segmentations in order




In [9]:

###################
# INPUT LY
###################

raw_lyso =  img_2D[1].copy()
LY_object, out_p =  infer_LYSOSOMES(raw_lyso, CY_object, out_p) 

intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [10]:

###################
# INPUT MT
###################
raw_mito    = img_2D[2].copy()

MT_object, out_p =  infer_MITOCHONDRIA(raw_mito, CY_object, out_p) 


In [11]:

###################
# INPUT GOLGI
###################
raw_golgi    = img_2D[3].copy()

GL_object, out_p =  infer_GOLGI(raw_golgi, CY_object, out_p) 



In [12]:

###################
# INPUT PEROXISOME
###################
raw_peroxi    = img_2D[4].copy()

PO_object, out_p =  infer_PEROXISOME(raw_peroxi, CY_object, out_p) 

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [13]:

###################
# INPUT ER
###################
raw_er  = img_2D[5].copy()

ER_object, out_p =  infer_ENDOPLASMIC_RETICULUM(raw_er, CY_object, out_p) 


###################
# INPUT LIPID DRP
###################
raw_lipid   = img_2D[6].copy()

LD_object, out_p =  infer_LIPID_DROPLET(raw_lipid, CY_object, out_p) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


## View

In [37]:
viewer = napari.Viewer()

In [38]:
viewer.add_image(
    raw_lyso,
    scale=scale,
    colormap='cyan', 
    blending='additive'
)

viewer.add_image(
    LY_object,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    raw_mito,
    scale=scale,
    colormap='green', 
    blending='additive'
)

viewer.add_image(
    MT_object,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    raw_golgi,
    scale=scale,
    colormap='yellow', 
    blending='additive'
)

viewer.add_image(
    GL_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    raw_peroxi,
    scale=scale,
    colormap='bop orange', 
    blending='additive'
)

viewer.add_image(
    PO_object,
    scale=scale,
    blending='additive')


viewer.add_image(
    raw_er,
    scale=scale,
    colormap='red', 
    blending='additive'
)

viewer.add_image(
    ER_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    raw_lipid,
    scale=scale,
    colormap='red', 
    blending='additive'
)

viewer.add_image(
    LD_object,
    scale=scale,
    blending='additive')


<Image layer 'LD_object' at 0x194375f70>

In [39]:

viewer.add_image(
    nuclei,
    scale=scale,
    colormap='blue', 
    blending='additive'
)

viewer.add_image(
    NU_object,
    scale=scale,
    blending='additive')


<Image layer 'NU_object' at 0x194acb670>


Pack into an OME.tiff with the same 

In [34]:

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 4,
    "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
    "nuc_ch" : 0,
    "optimal_Z": optimal_Z,
})
ch_to_agg = (1,2,3,4,5,6)
nuc_ch = 0

for i,target_file in enumerate(img_file_list):

    if i>0:
        break 

    bioim_image = read_input_image(target_file)
    img_data = bioim_image.image
    raw_meta_data = bioim_image.raw_meta
    ome_types = []
    meta_dict = bioim_image.meta

    # get some top-level info about the RAW data
    channel_names = meta_dict['name']
    img = meta_dict['metadata']['aicsimage']
    scale = meta_dict['scale']
    channel_axis = meta_dict['channel_axis']

    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    # make sure we have removed Z
    if len(scale)>2:
        scale = scale[1:]

    img_2D = img_data[:,[optimal_Z],:,:].copy()

    raw_nuclei = img_2D[0].copy()
    NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 

    raw_soma = (4. * img_2D[1].copy() + 
                                1. * img_2D[5].copy() + 
                                1. * img_2D[7].copy() )

    SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, out_p) 
    CY_object =  infer_CYTOSOL(SO_object, NU_object) 

    nuclei  = apply_mask(raw_nuclei, SO_object)
    NU_object  = apply_mask(NU_object, SO_object)

    raw_lyso =  img_2D[1].copy()
    LY_object, out_p =  infer_LYSOSOMES(raw_lyso, CY_object, out_p) 

    raw_mito    = img_2D[2].copy()
    MT_object, out_p =  infer_MITOCHONDRIA(raw_mito, CY_object, out_p) 

    raw_golgi    = img_2D[3].copy()
    GL_object, out_p =  infer_GOLGI(raw_golgi, CY_object, out_p) 

    raw_peroxi    = img_2D[4].copy()
    PO_object, out_p =  infer_PEROXISOME(raw_peroxi, CY_object, out_p) 

    raw_er  = img_2D[5].copy()
    ER_object, out_p =  infer_ENDOPLASMIC_RETICULUM(raw_er, CY_object, out_p) 

    raw_lipid   = img_2D[6].copy()
    LD_object, out_p =  infer_LIPID_DROPLET(raw_lipid, CY_object, out_p) 




The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [ ]:
##  need to save: 

# output_path, list_of_files

In [35]:
optimal_Z

12

In [33]:
napari.view_image(img_data)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 383.5, 383.5), zoom=1.0551432291666665, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 8.0, 1.0), (0.0, 16.0, 1.0), (0.0, 768.0, 1.0), (0.0, 768.0, 1.0)), current_step=(4, 8, 384, 384), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'img_data' at 0x19e4c4d60>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, sh